# Set up Essentials

In [0]:
from google.colab import drive , files
import os, os.path
import pandas as pd
from pandas.io.json import json_normalize 
import json
import requests
import numpy as np

## **Mount Drive and Operate from within said folder**

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [6]:
%cd /content/drive/'My Drive'/Colab_Notebooks/mzalendo

/content/drive/My Drive/Colab_Notebooks/mzalendo


In [0]:
persons = "http://info.mzalendo.com/media_root/popolo_json/persons.json"
organization = "http://info.mzalendo.com/media_root/popolo_json/organizations.json"
memberships = "http://info.mzalendo.com/media_root/popolo_json/memberships.json"
pombola = "http://info.mzalendo.com/media_root/popolo_json/memberships.json"
candidates = "https://docs.google.com/spreadsheets/d/e/2PACX-1vTR3hgRnOrPi6Opkr3Z622_onNYEtzds51TxqS1-VBEcDqP8O0nLjr1XwRkQYCGLrusNL6nJLyxtQbe/pub?gid=671749755&single=true&output=csv"
elected = "https://docs.google.com/spreadsheets/d/e/2PACX-1vTR3hgRnOrPi6Opkr3Z622_onNYEtzds51TxqS1-VBEcDqP8O0nLjr1XwRkQYCGLrusNL6nJLyxtQbe/pub?gid=655626479&single=true&output=csv"


## Work from file with nested jsons'

In [0]:
with open(pombola) as data_file:    
    data = json.load(data_file) 

# data = json_normalize("pombola.json")

In [0]:
df = json_normalize(data)

In [0]:
df.head()

,areas,events,organizations,persons,posts
0,"[{'area_type': 'CON', 'id': 'mapit:4812', 'ide...","[{'classification': 'legislative period', 'end...","[{'category': 'other', 'classification': 'Unkn...","[{'contact_details': [], 'id': 'core_person:78...",[]


In [0]:
df.columns

Index(['areas', 'events', 'organizations', 'persons', 'posts'], dtype='object')

In [0]:
len(df['persons'][0])

13267

In [0]:
df_P = pd.DataFrame(df['persons'][0])

In [0]:
df_P.head(3)

,contact_details,id,identifiers,images,memberships,name,other_names,sort_name,birth_date,gender,death_date,summary,email,biography,national_identity,family_name,given_name,honorific_suffix,honorific_prefix
0,[],core_person:7848,"[{'identifier': 'abbey-gedow-abdikadir', 'sche...",[{'url': 'http://info.mzalendo.com/media_root/...,"[{'area': None, 'end_date': '2013-03-04', 'id'...",Abbey Gedow Abdikadir,[],Abdikadir,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[],core_person:8251,"[{'identifier': 'al-amin-ahmed', 'scheme': 'po...",[{'url': 'http://info.mzalendo.com/media_root/...,"[{'id': 'core_position:21297', 'identifiers': ...",Al Amin Ahmed,[],Ahmed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[],core_person:4435,"[{'identifier': 'abdi-ahmed-ali', 'scheme': 'p...",[{'url': 'http://info.mzalendo.com/media_root/...,"[{'id': 'core_position:13754', 'identifiers': ...",Abdi Ahmed Ali,[],Ali,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Collect All Candidates data

In [0]:
all = pd.read_csv(candidates)
elected = pd.read_csv(elected)

In [0]:
all.head(3)

,name,party,party-id,p-abbrv,gender,position,county,county-id,constituency,constituency-id,ward,ward-id
0,JOHN EKURU LONGOGGY AUKOT,THIRDWAY ALLIANCE KENYA,76,TAK,Male,president,NaN,NaN,NaN,NaN,NaN,NaN
1,MOHAMED ABDUBA DIDA,ALLIANCE FOR REALCHANGE,55,ARK,Male,president,NaN,NaN,NaN,NaN,NaN,NaN
2,SHAKHALAGA KHWA JIRONGO,UNITED DEMOCRATIC PARTY,64,UDP,Male,president,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
all.shape

(14758, 12)

## Isolate Elected legislators

In [0]:
all['elected'] = np.where(all['name'].isin(elected['name'].str.upper()),"elected-2017", "not-elected")


In [0]:
all.head(3)

,name,party,party-id,p-abbrv,gender,position,county,county-id,constituency,constituency-id,ward,ward-id,elected
0,JOHN EKURU LONGOGGY AUKOT,THIRDWAY ALLIANCE KENYA,76,TAK,Male,president,NaN,NaN,NaN,NaN,NaN,NaN,not-elected
1,MOHAMED ABDUBA DIDA,ALLIANCE FOR REALCHANGE,55,ARK,Male,president,NaN,NaN,NaN,NaN,NaN,NaN,not-elected
2,SHAKHALAGA KHWA JIRONGO,UNITED DEMOCRATIC PARTY,64,UDP,Male,president,NaN,NaN,NaN,NaN,NaN,NaN,not-elected


In [0]:
# all.to_csv("./ke_legislators.csv")

# Persons Section

Access the json file

In [0]:
data_p = pd.read_json(persons)

In [115]:
data_p.head(3)

,_id,contact_details,id,identifiers,images,name,other_names,sort_name,birth_date,gender,death_date,summary,email,biography,national_identity,family_name,given_name,honorific_suffix,honorific_prefix
0,core_person:7848,[],core_person:7848,"[{'identifier': 'abbey-gedow-abdikadir', 'sche...",[{'url': 'http://info.mzalendo.com/media_root/...,Abbey Gedow Abdikadir,[],Abdikadir,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,core_person:8251,[],core_person:8251,"[{'identifier': 'al-amin-ahmed', 'scheme': 'po...",[{'url': 'http://info.mzalendo.com/media_root/...,Al Amin Ahmed,[],Ahmed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,core_person:4435,[],core_person:4435,"[{'identifier': 'abdi-ahmed-ali', 'scheme': 'p...",[{'url': 'http://info.mzalendo.com/media_root/...,Abdi Ahmed Ali,[],Ali,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
p_columns = ['id', 'name','contact_details', 'images','other_names', 'birth_date', 'gender', 'death_date',
             'summary', 'email', 'biography', 'national_identity']

data_p = data_p[p_columns]

In [117]:
data_p.head(3)

,id,name,contact_details,images,other_names,birth_date,gender,death_date,summary,email,biography,national_identity
0,core_person:7848,Abbey Gedow Abdikadir,[],[{'url': 'http://info.mzalendo.com/media_root/...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,core_person:8251,Al Amin Ahmed,[],[{'url': 'http://info.mzalendo.com/media_root/...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,core_person:4435,Abdi Ahmed Ali,[],[{'url': 'http://info.mzalendo.com/media_root/...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN


Clean up *id* and *images*  columns

In [0]:
def clean_id(x):
  id = x.split(":")[1]
  return id

def get_img_url(x):
  if pd.notnull(x):
    url = x[0].get('url')
    return url


In [0]:
# Access the ID's only
data_p['id'] = data_p['id'].apply(lambda x: clean_id(x))
# get url from dictionary
data_p['images'] = data_p['images'].apply(lambda x: get_img_url(x))

In [0]:
# data_p.to_csv('ke_persons.csv')